# Tensorflow for eigenvalue problem

Testing audograd for eigenvalue problem. Or PMM(Parametric Matrix Model) 

Suppose True problem have eigenvalues $e_t(c)$.
Prepare random matrix
$M(c) = M_0 + c M_1$, and solve eigenvalue problem $M(c) v = e v$. 

The goal is to reduce the difference $|e(c)-e_t(c)|^2$
by updating matrix $M_0$ and $M_1$. In other words, obtaining 
$M(c)$ which simulate the original problem. 


In [1]:
#========== import packages=============== 
import numpy as np
import tensorflow as tf
#tf.get_logger().setLevel('ERROR')
#tf.autograph.set_verbosity(1)

In [2]:
#============set up matrix====================
# target true matrix
c= tf.constant(0.5,dtype=tf.complex128)
s0 = tf.constant( [ [1.0,0.0],[0.0,1.0]],dtype=tf.complex128)
sx = tf.constant( [ [0.0,1.0],[1.0,0.0]],dtype=tf.complex128)
sy = tf.constant( [ [0.0+0j,-1j],[1j,0.0+0j]],dtype=tf.complex128)
sz = tf.constant( [ [1.0,0.0],[0.0,-1.0]],dtype=tf.complex128)

def mm_t(c):
    return sx+ sz*c 

e_t,v_t = tf.linalg.eig(mm_t(0.5))

#------initial matrix
learning_rate=tf.constant(0.01,dtype=tf.complex128)
var = tf.Variable([0.1,0.5,0.1,1.0],dtype=tf.complex128) # true is (0,1,0,0.5)

def mm_var(var):
    return s0*var[0]+sx*var[1]+sy*var[2]+sz*var[3]

e,v = tf.linalg.eig(mm_var(var))

print('mm_t :{} '.format(mm_t(0.5)))
print('e_t :{} '.format(e_t))
print('mm(0) :{} '.format(mm_var(var)))
print('e(0) :{} '.format(e))


mm_t :[[ 0.5+0.j  1. +0.j]
 [ 1. +0.j -0.5+0.j]] 
e_t :[ 1.11803399+0.j -1.11803399+0.j] 
mm(0) :[[ 1.1+0.j   0.5-0.1j]
 [ 0.5+0.1j -0.9+0.j ]] 
e(0) :[-1.02249722+6.30407896e-18j  1.22249722-6.30407896e-18j] 


In [3]:
#----compute loss and record tape 
def one_step(var):
    with tf.GradientTape() as tape:
        #print('mat:\n {}'.format(mm.numpy()))     
        e,v = tf.linalg.eig(mm_var(var))
        #print('e(0) :{} '.format(e))
        loss = tf.math.reduce_sum(tf.math.abs(e-e_t)**2)
        print('loss: {}'.format(loss.numpy()))     
        #---update
    dvar = tape.gradient(loss,var)
    var = tf.Variable(var - dvar*learning_rate,dtype=tf.complex128)
    return var 

In [4]:
for i in range(10):
    var = one_step(var)

loss: 10.059960159204447
loss: 9.27125928272282
loss: 8.544392554957348
loss: 7.874512178648692
loss: 7.257150423842635
loss: 6.688189830613372
loss: 6.163835747893286
loss: 5.680591025258451
loss: 5.235232688878189
loss: 4.824790446070139


In [5]:
e,v = tf.linalg.eig(mm_var(var))
print(var)
print(e)
print(e_t)

<tf.Variable 'Variable:0' shape=(4,) dtype=complex128, numpy=
array([0.06648326-8.98145931e-19j, 0.16549898-1.66533454e-18j,
       0.0330998 +8.88178420e-18j, 0.33099795+1.32230012e-18j])>
tf.Tensor([-0.30506101+5.00740002e-18j  0.43802754+2.35564807e-18j], shape=(2,), dtype=complex128)
tf.Tensor([ 1.11803399+0.j -1.11803399+0.j], shape=(2,), dtype=complex128)


### It is not clear how to use Adam optimizer in tensorflow

# PyTorch test

## test with simple gradient

In [6]:
#-----testing pytorch
import torch
from torch import FloatTensor
from torch.autograd import Variable

w = Variable(FloatTensor([[4,2],[2,1]]),requires_grad=True)
ee,vv = torch.linalg.eig(w)
e_t = torch.Tensor([ 1.0,-1.0]) # target eigenvalues

def get_loss(ee,e_t):
    return (torch.abs(ee-e_t)**2).sum()
loss = get_loss(ee,e_t)
print(loss)

loss.backward()
dw = w.grad

for i in range(100):
    w =  Variable(w-dw*0.01,requires_grad=True) 
    ee,vv = torch.linalg.eig(w)
    loss = get_loss(ee,e_t)
    #print(loss)
    loss.backward()
    dw = w.grad
print(loss) 
print(ee,e_t)

tensor(17., grad_fn=<SumBackward0>)
tensor(0.2990, grad_fn=<SumBackward0>)
tensor([ 1.5305+0.j, -0.8674+0.j], grad_fn=<LinalgEigBackward0>) tensor([ 1., -1.])


## test with Adam optimizer

In [7]:
w = Variable(FloatTensor([[4,2],[2,1]]),requires_grad=True) # prepare variable
optim = torch.optim.Adam([w],lr=0.01)
#optim = torch.optim.SGD([w],lr=0.01)
ee,vv = torch.linalg.eig(w)
loss = get_loss(ee,e_t) 

In [8]:
#--test model construction in Pytorch
print(loss)
for i in range(10):
    ee,vv = torch.linalg.eig(w)
    loss = get_loss(ee,e_t)
    #print(loss)
    optim.zero_grad()
    loss.backward()
    optim.step()
print(loss)    
print(ee,e_t)

tensor(17., grad_fn=<SumBackward0>)
tensor(15.6997, grad_fn=<SumBackward0>)
tensor([ 4.8388+0.j, -0.0186+0.j], grad_fn=<LinalgEigBackward0>) tensor([ 1., -1.])


## test with the same problem as tensorflow 

In [9]:
c= 0.5
s0 = torch.tensor( [ [1.0,0.0],[0.0,1.0]])
sx = torch.tensor( [ [0.0,1.0],[1.0,0.0]])
sy = torch.tensor( [ [0.0+0j,-1j],[1j,0.0+0j]],dtype=torch.complex64)
sz = torch.tensor( [ [1.0,0.0],[0.0,-1.0]])

def mm_t(c):
    return sx+ sz*c 

e_t,v_t = torch.linalg.eig(mm_t(0.5))

#------initial matrix
learning_rate=torch.tensor(0.01)
var = Variable(torch.tensor([0.1,0.5,0.1,1.0]),requires_grad=True) # true is (0,1,0,0.5)
optim = torch.optim.Adam([var])

def mm_var(var):
    return s0*var[0]+sx*var[1]+sy*var[2]+sz*var[3]

def get_loss(ee,e_t):
    return (torch.abs(ee-e_t)**2).sum()

ee,vv = torch.linalg.eig(mm_var(var))

In [1]:
print(get_loss(ee,e_t))  
for i in range(100):
    ee,vv = torch.linalg.eig(mm_var(var))
    loss = get_loss(ee,e_t)
    #print(loss)
    optim.zero_grad()
    loss.backward()
    optim.step()
print(loss)    

NameError: name 'get_loss' is not defined

Note that two matrices , mm_t and mm_var is not necessarily be the same matrices.
They only share the eigenvalues and could be related with unitary transformation. 


In [2]:
print(torch.linalg.eig(mm_var(var)))
print(torch.linalg.eig(mm_t(0.5)))

NameError: name 'torch' is not defined